# 01. Perform simulations

In [ ]:
from orix.crystal_map import Phase
from orix.sampling import get_sample_reduced_fundamental
from diffsims.generators.simulation_generator import SimulationGenerator
from diffpy.structure import Lattice, Structure, Atom
from parse_pdb_with_scale_remove_h import parse_pdb_with_scale

pdbfile = "/Users/xiaodong/Desktop/UOX-simulations/UOX.pdb"

cell, sg_sym, atoms = parse_pdb_with_scale(pdbfile, remove_hydrogens=True, include_occupancy=True)

lattice = Lattice(*cell)

# Define the phase using less atoms than in the pdb for computational reasons
atoms = [Atom("C", (0, 0, 0), 1.0),]

# Create the structure and corresponding phase
structure = Structure(atoms, lattice)
phase = Phase(space_group = 23, structure=structure)

# Sample orientations in the symmetry-reduced zone (resolution in degrees)
orientations = get_sample_reduced_fundamental(
    resolution=10,
    point_group=phase.point_group,
)

# Initialize the simulation generator (e.g., setting the accelerating voltage)
generator = SimulationGenerator(
        accelerating_voltage = 300,
        scattering_params = "lobato",
        precession_angle = 0,
        shape_factor_model = "lorentzian",
        approximate_precession = False,
        minimum_intensity = 1e-20,
        )

# Calculate 2D diffraction simulations using the provided phase and orientations
simulations = generator.calculate_diffraction2d(
    phase = phase,
    rotation = orientations,
    reciprocal_radius = 1/1.3,  # 1/Å (i.e. Å⁻¹)
    with_direct_beam = True,   # Include the direct beam
    max_excitation_error = 0.0005,
    shape_factor_width = None,
    debye_waller_factors = None,
    show_progressbar = True
)

# Plot the simulations (this may include an interactive slider)
simulations.plot(in_plane_angle=0, calibration = 0.0015090274190359715)

Symmetry (4,) 222
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


 36%|███▌      | 61/171 [00:04<00:07, 14.66it/s]

In [ ]:
# Make sure plots display inline
%matplotlib inline
import os
import ipywidgets as widgets
from ipywidgets import interact
from process_simulation import process_simulation
from helper_functions import copy_h5_file, get_next_simulation_folder, load_h5_data, view_image

# Specify the input HDF5 file
input_filename = "/Users/xiaodong/Desktop/UOX-simulations/UOX1_sub_radial_backgrounds.h5"
base_dir = os.path.dirname(input_filename)

# 1. Create a new simulation folder
new_sim_folder = get_next_simulation_folder(base_dir)

# 2. Copy the original file to the new simulation folder
output_filename = os.path.join(new_sim_folder, "sim.h5")
copy_h5_file(input_filename, output_filename)

# 3. Process the simulation images.
# Note: 'simulations' should be defined (or imported) before calling process_simulation.
process_simulation(output_filename,
                   simulations, 
                   intensity_scale=1000,
                   calibration=0.0015090274190359715, 
                   sigma=1.5, 
                   fast_clip_threshold=0.1)

# 4. Load the processed data for interactive visualization
images_arr, angles_arr = load_h5_data(output_filename)

# 5. Set up the interactive viewer
interact(lambda index: view_image(index, images_arr, angles_arr), 
         index=widgets.IntSlider(min=0, max=images_arr.shape[0]-1, step=1, value=0))


Processing images: 100%|██████████| 100/100 [00:01<00:00, 63.11it/s]


Processing complete. Updated file saved at: /Users/xiaodong/Desktop/UOX-simulations/simulation-28/sim.h5


interactive(children=(IntSlider(value=0, description='index', max=99), Output()), _dom_classes=('widget-intera…

<function __main__.<lambda>(index)>